In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [3]:
#Paso el timestamp del df 
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['year'] = events_df['timestamp'].dt.year
events_df['month'] = events_df['timestamp'].dt.month
events_df['day'] = events_df['timestamp'].dt.day
events_df['weekday'] = events_df['timestamp'].dt.day_name()
events_df['hour'] = events_df['timestamp'].dt.hour

## generacion de features

In [286]:
features_df = pd.DataFrame()

In [287]:
features_df['person'] = events_df.person.unique()

In [288]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Quincenas

In [ ]:
features_df['q1'] = ((events['mes']==1) & (events['dia']>1) & (events['dia']<16))
events['q2'] = ((events['mes']==1) & (events['dia']>15) & (events['dia']<32))
events['q3'] = ((events['mes']==2) & (events['dia']>1) & (events['dia']<16))
events['q4'] = ((events['mes']==2) & (events['dia']>15) & (events['dia']<29))
events['q5'] = ((events['mes']==3) & (events['dia']>1) & (events['dia']<16))
events['q6'] = ((events['mes']==3) & (events['dia']>15) & (events['dia']<32))
events['q7'] = ((events['mes']==4) & (events['dia']>1) & (events['dia']<16))
events['q8'] = ((events['mes']==4) & (events['dia']>15) & (events['dia']<31))
events['q9'] = ((events['mes']==5) & (events['dia']>1) & (events['dia']<16))
events['q10'] = ((events['mes']==5) & (events['dia']>15) & (events['dia']<32))

In [ ]:
features_df.eventos_enero = events_df.groupby(['person','month']).count()

In [10]:
aux = events_df.groupby(['person','month']).count().unstack()
aux.loc[ aux.month == 1]

AttributeError: 'DataFrame' object has no attribute 'month'

In [14]:
ct = pd.crosstab(events_df.person, events_df.month)
df = pd.DataFrame(data=ct)

In [15]:
df.head()

month,1,2,3,4,5
person,,,,,
0008ed71,0,0,0,0,6
00091926,0,0,0,0,448
00091a7a,0,0,10,0,0
000ba417,0,0,0,0,206
000c79fe,0,0,0,0,17


In [21]:
features_df = features_df.join(df, on='person')

In [40]:
#modelos por mes

ct = pd.crosstab(events_df.person, events_df.month, events_df.model, aggfunc='count')
df = pd.DataFrame(data=ct)

In [48]:
df.rename(columns={1:'modelos_enero', 2:'modelos_febrero', 3:'modelos_marzo', 4:'modelos_abril',
                   5:'modelos_mayo'}, inplace=True)
df.columns

Index(['modelos_enero', 'modelos_febrero', 'modelos_marzo', 'modelos_abril',
       'modelos_mayo'],
      dtype='object', name='month')

In [49]:
features_df = features_df.join(df, on='person')

## Candidad de eventos totales de cada usuario

In [275]:
df = events_df.groupby('person').event.count()

In [276]:
features_df = features_df.join(df, on='person')

In [277]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad de cada evento

In [278]:
df = events_df.groupby('person').event.value_counts()

In [279]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [280]:
df['ach*bl'] = (df['ad campaign hit'])*(df['brand listing'])
#df['ach*ch'] = (df['ad campaign hit'])*(df['checkout'])
#df['ach*co'] = (df['ad campaign hit'])*(df['conversion'])
df['ach*gl'] = (df['ad campaign hit'])*(df['generic listing'])
#df['ach*ld'] = (df['ad campaign hit'])*(df['lead'])
df['ach*seh'] = (df['ad campaign hit'])*(df['search engine hit'])
df['ach*sp'] = (df['ad campaign hit'])*(df['searched products'])
#df['ach*st'] = (df['ad campaign hit'])*(df['staticpage'])
df['ach*vp'] = (df['ad campaign hit'])*(df['viewed product'])
df['ach*vs'] = (df['ad campaign hit'])*(df['visited site'])

In [281]:
df['ch*bl'] = (df['checkout'])*(df['brand listing'])
df['ch*co'] = (df['checkout'])*(df['conversion'])
df['ch*gl'] = (df['checkout'])*(df['generic listing'])
#df['ch*ld'] = (df['checkout'])*(df['lead'])
#df['ch*seh'] = (df['checkout'])*(df['search engine hit'])
#df['ch*sp'] = (df['checkout'])*(df['searched products'])
#df['ch*st'] = (df['checkout'])*(df['staticpage'])
df['ch*vp'] = (df['checkout'])*(df['viewed product'])
df['ch*vs'] = (df['checkout'])*(df['visited site'])

In [282]:
df['vp*bl'] = (df['viewed product'])*(df['brand listing'])
df['vp*co'] = (df['viewed product'])*(df['conversion'])
df['vp*gl'] = (df['viewed product'])*(df['generic listing'])
#df['vp*ld'] = (df['viewed product'])*(df['lead'])
df['vp*seh'] = (df['viewed product'])*(df['search engine hit'])
df['vp*sp'] = (df['viewed product'])*(df['searched products'])
#df['vp*st'] = (df['viewed product'])*(df['staticpage'])
df['vp*vs'] = (df['viewed product'])*(df['visited site'])

In [283]:
df['sp*bl'] = (df['searched products'])*(df['brand listing'])
#df['sp*co'] = (df['searched products'])*(df['conversion'])
df['sp*gl'] = (df['searched products'])*(df['generic listing'])
#df['sp*ld'] = (df['searched products'])*(df['lead'])
df['sp*seh'] = (df['searched products'])*(df['search engine hit'])
#df['sp*st'] = (df['searched products'])*(df['staticpage'])
df['sp*vs'] = (df['searched products'])*(df['visited site'])

In [284]:
df['bl*co'] = (df['brand listing'])*(df['conversion'])
df['bl*ch'] = (df['brand listing'])*(df['checkout'])
df['bl*seh'] = (df['brand listing'])*(df['search engine hit'])
df['bl*vs'] = (df['brand listing'])*(df['visited site'])
df['bl*st'] = (df['brand listing'])*(df['staticpage'])

In [65]:
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,ach*bl,ach*ch,ach*co,ach*gl,ach*ld,ach*seh,ach*sp,ach*st,ach*vp,ach*vs
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,...,375.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,5580.0,510.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,...,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,...,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [289]:
features_df = features_df.join(df, on='person')

In [290]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

In [183]:
features_df.head()

,person,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,sp*bl,sp*co,sp*gl,sp*ld,sp*seh,sp*st,sp*vs
0,4886f805,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,4.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1,ad93850f,10.0,15.0,1.0,NaN,7.0,NaN,7.0,NaN,NaN,20.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0297fc1e,29.0,4.0,7.0,NaN,21.0,1.0,NaN,6.0,NaN,404.0,95.0,24.0,NaN,126.0,6.0,NaN,NaN,570.0
3,2d681dd8,1.0,5.0,1.0,NaN,1.0,NaN,2.0,1.0,NaN,13.0,2.0,5.0,NaN,1.0,NaN,2.0,NaN,2.0
4,cccea85e,15.0,7.0,1.0,NaN,20.0,NaN,26.0,1.0,5.0,739.0,22.0,7.0,NaN,20.0,NaN,26.0,5.0,22.0


## Cantidad de returnings

In [ ]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [ ]:
features_df = features_df.join(df, on='person')

## Tiempo total 

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [ ]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_total'})

In [ ]:
features_df[['person', 'tiempo_total']].head()

In [ ]:
features_df.tiempo_total = pd.to_numeric(features_df.tiempo_total, downcast='float')
features_df.tiempo_total.head()

## Tiempo entre new y returning

In [ ]:
df_news = events_df.loc[events_df.new_vs_returning == 'New']
df_news[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df = (df_returnings.groupby('person').timestamp.min() - df_news.groupby('person').timestamp.min())
df = df.fillna(0)
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_new_returning'})

In [ ]:
features_df[['person', 'tiempo_new_returning']].head()

In [ ]:
features_df.tiempo_new_returning = pd.to_numeric(features_df.tiempo_new_returning, downcast='float')
features_df.tiempo_new_returning.head()

## Mean entre tiempos de returnings

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings.groupby('person').head()

## Channel mas frecuente

In [ ]:
def agg_mas_frecuente(x):
    return (x.value_counts().index[0] if(len(x.value_counts())>0) else np.nan)

In [ ]:
df = events_df.groupby('person').channel.agg(lambda x: agg_mas_frecuente(x))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.columns

In [ ]:
features_df.rename(columns={'channel':'channel_frecuente'}, inplace=True)

In [ ]:
df.value_counts().idxmax()

In [ ]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [ ]:
features_df.channel_frecuente.isna().value_counts()

In [ ]:
features_df[['person', 'channel_frecuente']].head()

## Device mas frecuente

In [ ]:
df = events_df.groupby('person').device_type.agg(lambda x: agg_mas_frecuente(x))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'device_type':'device_frecuente'}, inplace=True)

In [ ]:
df.value_counts().idxmax()

In [ ]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [ ]:
features_df.head()

## Support para eventos

In [ ]:
ct = pd.crosstab(events_df.person, events_df.event)
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.0, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [ ]:
result = columna_por_persona(ct)

In [ ]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support    

In [ ]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value  

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas  

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'event_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Support para el storage

In [ ]:
ct = pd.crosstab(events_df.person, events_df.storage)

In [ ]:
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.0, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'storage_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Support para color

In [ ]:
ct = pd.crosstab(events_df.person, events_df.color)
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'color_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Support para condition

In [ ]:
ct = pd.crosstab(events_df.person, events_df.condition)
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 
frequent_itemsets = apriori(ct, min_support=0.0, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
result = columna_por_persona(ct)

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'condition_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Encoding de las variables categoricas

In [ ]:
import category_encoders as ce

## Channel frecuente

In [ ]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

## Device frecuente

In [ ]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [100]:
features_df.columns

Index([         'person',                 1,                 2,
                       3,                 4,                 5,
         'modelos_enero', 'modelos_febrero',   'modelos_marzo',
         'modelos_abril',    'modelos_mayo',        'cant_ach',
               'cant_bl',         'cant_ch',         'cant_cv',
               'cant_gl',          'cant_l',        'cant_seh',
              'cant_s_p',         'cant_sp',         'cant_vp',
               'cant_vs',          'ach*bl',          'ach*ch',
                'ach*co',          'ach*gl',          'ach*ld',
               'ach*seh',          'ach*sp',          'ach*st',
                'ach*vp',          'ach*vs'],
      dtype='object')

In [103]:
features_df = features_df.drop(columns=[1, 2, 3, 'modelos_enero', 'modelos_febrero', 'modelos_marzo', 'cant_l', 'ach*ld', 'ach*st'])

In [118]:
features_df = features_df.drop(columns=['cant_sp', 'cant_cv', 'ach*co'])

## Leo los labels

In [24]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [291]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 39)

In [292]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 38)

##  Random forest

In [ ]:
#Uso https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

In [ ]:
#Separo features de labels

In [293]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])


In [294]:
train_df.fillna(0, inplace=True)

In [295]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [296]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [297]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

In [ ]:
#Encoding de las vbles categoricas

#Channel frecuente
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [ ]:
#Encoding de las vbles categoricas

#Device frecuente
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

In [ ]:
#Aca ver lo del baseline

In [298]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [299]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [35]:
from sklearn.metrics import roc_auc_score

In [300]:
roc_auc_score(y_test,predictions)

0.7566585956416464

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [301]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_ch              Importance: 0.08
Variable: vp*vs                Importance: 0.06
Variable: cant_vp              Importance: 0.05
Variable: ach*vp               Importance: 0.05
Variable: vp*bl                Importance: 0.05
Variable: vp*gl                Importance: 0.05
Variable: vp*seh               Importance: 0.05
Variable: ach*vs               Importance: 0.04
Variable: bl*vs                Importance: 0.04
Variable: cant_gl              Importance: 0.03
Variable: cant_vs              Importance: 0.03
Variable: ach*bl               Importance: 0.03
Variable: ach*gl               Importance: 0.03
Variable: ach*seh              Importance: 0.03
Variable: ch*vp                Importance: 0.03
Variable: vp*sp                Importance: 0.03
Variable: bl*seh               Importance: 0.03
Variable: cant_ach             Importance: 0.02
Variable: cant_bl              Importance: 0.02
Variable: cant_seh             Importance: 0.02
Variable: ach*sp               Importanc

In [ ]:
#Saco las columnas con importancia menor o igual a 0.05 de las cant y recalculo
new_train_df = train_df.drop(columns=['tiempo_total'])

In [ ]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(new_train_df.drop(columns=['person']).columns)

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(new_train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [ ]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

In [ ]:
#Encoding de las vbles categoricas

#Channel frecuente
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [ ]:
#Encoding de las vbles categoricas

#Device frecuente
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [ ]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [ ]:
roc_auc_score(y_test,predictions)

In [ ]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

## Prediccion

In [84]:
to_predict.drop(columns='person').columns

Index([                1,                 2,                 3,
                       4,                 5,   'modelos_enero',
       'modelos_febrero',   'modelos_marzo',   'modelos_abril',
          'modelos_mayo',        'cant_ach',         'cant_bl',
               'cant_ch',         'cant_cv',         'cant_gl',
                'cant_l',        'cant_seh',        'cant_s_p',
               'cant_sp',         'cant_vp',         'cant_vs',
                'ach*bl',          'ach*ch',          'ach*co',
                'ach*gl',          'ach*ld',         'ach*seh',
                'ach*sp',          'ach*st',          'ach*vp',
                'ach*vs'],
      dtype='object')

In [ ]:
#to_predict = to_predict.drop(columns=['tiempo_total'])

In [85]:
to_predict.fillna(0, inplace=True)

In [ ]:
to_predict.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').channel_frecuente))

In [ ]:
to_predict.device_frecuente = te_device_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').device_frecuente))

In [86]:
preds_posta = rf.predict(to_predict.drop(columns=['person']))

In [87]:
preds_posta

array([0.        , 0.13      , 0.03      , ..., 0.05201923, 0.026     ,
       0.026     ])

## XGBoost

In [95]:
from xgboost import XGBRegressor
import xgboost as xgb

In [ ]:
X_train.columns

In [170]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', booster='dart',
                          base_score=0.1, colsample_bylevel=0.9,
                        colsample_bytree = 0.1, learning_rate = 0.15,
                        max_depth = 13, alpha = 20, n_estimators = 80)

In [302]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.1, gamma=0, learning_rate=0.15, max_delta_step=0,
       max_depth=13, min_child_weight=1, missing=None, n_estimators=80,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [303]:
preds = xg_reg.predict(X_test)

In [304]:
roc_auc_score(y_test,preds)

0.7978961852251621

## CSV a publicar

In [88]:
to_publish = pd.DataFrame()

In [89]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [90]:
to_publish['label'] = preds_posta
to_publish.shape

(19415, 2)

In [91]:
to_publish.head()

,person,label
0,4886f805,0.00
1,0297fc1e,0.13
2,2d681dd8,0.03
3,cccea85e,0.11
4,4c8a8b93,0.02


In [92]:
to_publish.loc[ to_publish.label > 1 ]

,person,label


In [93]:
to_publish.loc[ to_publish.label < 0 ]

,person,label


In [94]:
to_publish.to_csv('28_11_4.csv', index=False)